In [1]:
library(Seurat)
library(Signac)
library(genomation)
library(GenomicRanges)
library(BSgenome.Hsapiens.UCSC.hg38)
library(EnsDb.Hsapiens.v86)
library(Matrix)
library(parallel)
library(foreach)

Loading required package: SeuratObject

Loading required package: sp

‘SeuratObject’ was built under R 4.3.2 but the current version is
4.3.3; it is recomended that you reinstall ‘SeuratObject’ as the ABI
for R may have changed

‘SeuratObject’ was built with package ‘Matrix’ 1.6.3 but the current
version is 1.6.5; it is recomended that you reinstall ‘SeuratObject’ as
the ABI for ‘Matrix’ may have changed


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect


Loading required package: grid

Warning message:
“replacing previous import ‘Biostrings::pattern’ by ‘grid::pattern’ when loading ‘genomation’”
Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following object is masked from ‘package:SeuratObject’:

    intersect


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDupl

Specify file path

In [2]:
path.matrix.atac_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/240508/sc-E2G/test/results_K562_Xu/K562/Kendall/atac_matrix.csv.gz"
path.matrix.rna_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/K562_Xu/1.prepare_data/1.seurat_pipeline.240507/rna_count_matrix.csv.gz"
dir.output = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/K562_Xu/3.Genome_wide_prediction/Signac.default/Signac.240611/"

Import ATAC matrix

In [3]:
matrix.atac = read.csv(path.matrix.atac_count,
                       row.names = 1,
                       check.names = F)
matrix.atac = Matrix(as.matrix(matrix.atac), sparse = TRUE)

Import RNA matrix

In [4]:
matrix.rna_count = read.csv(path.matrix.rna_count,
                            row.names = 1,
                            check.names = F)
matrix.rna_count = Matrix(as.matrix(matrix.rna_count), sparse = TRUE)
matrix.rna_count = matrix.rna_count[,colnames(matrix.atac)]

Create Seurat object, follow https://stuartlab.org/signac/articles/pbmc_multiomic

In [5]:
# get gene annotations for hg38
annotation <- GetGRangesFromEnsDb(ensdb = EnsDb.Hsapiens.v86)
seqlevels(annotation) <- paste0('chr', seqlevels(annotation))

Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warn

In [6]:
# create a Seurat object containing the RNA adata
obj.seurat <- CreateSeuratObject(
  counts = matrix.rna_count,
  assay = "RNA"
)

In [7]:
# create ATAC assay and add it to the object
obj.seurat[["ATAC"]] <- CreateChromatinAssay(
  counts = matrix.atac,
  sep = c("-", "-"),
  annotation = annotation
)

In [9]:
DefaultAssay(obj.seurat) <- "RNA"
obj.seurat <- SCTransform(obj.seurat)
obj.seurat <- RunPCA(obj.seurat)

Running SCTransform on assay: RNA

Running SCTransform on layer: counts

vst.flavor='v2' set. Using model with fixed slope and excluding poisson genes.

`vst.flavor` is set to 'v2' but could not find glmGamPoi installed.
Please install the glmGamPoi package for much faster estimation.
--------------------------------------------
install.packages('BiocManager')
BiocManager::install('glmGamPoi')
--------------------------------------------
Falling back to native (slower) implementation.


Variance stabilizing transformation of count matrix of size 28417 by 7821

Model formula is y ~ log_umi

Get Negative Binomial regression parameters per gene

Using 2000 genes, 5000 cells

Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
"iteration limit reached"
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
"iteration limit reached"
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = 

In [10]:
DefaultAssay(obj.seurat) <- "ATAC"
obj.seurat <- FindTopFeatures(obj.seurat, min.cutoff = 5)
obj.seurat <- RunTFIDF(obj.seurat)
obj.seurat <- RunSVD(obj.seurat)

Performing TF-IDF normalization

Running SVD

Scaling cell embeddings



Linking peaks to genes

In [11]:
DefaultAssay(obj.seurat) <- "ATAC"

# first compute the GC content for each peak
obj.seurat <- RegionStats(obj.seurat, genome = BSgenome.Hsapiens.UCSC.hg38)

In [ ]:
# link peaks to genes
obj.seurat <- LinkPeaks(
  object = obj.seurat,
  peak.assay = "ATAC",
  expression.assay = "SCT",
  genes.use = rownames(matrix.rna_count[rowSums(matrix.rna_count) > 0,])
)

Testing 24741 genes and 157600 peaks

Found gene coordinates for 14332 genes



Save results

In [14]:
dir.create(dir.output,recursive = T)
saveRDS(obj.seurat@assays$ATAC@links,
        paste(dir.output,"links.rds",sep = "/"))
obj.seurat@assays$ATAC@links

GRanges object with 19756 ranges and 5 metadata columns:
          seqnames              ranges strand |     score        gene
             <Rle>           <IRanges>  <Rle> | <numeric> <character>
      [1]     chr1      820756-1020123      * | 0.0525527        AGRN
      [2]     chr1      827285-1116361      * | 0.0532860    C1orf159
      [3]     chr1      940540-1000172      * | 0.0513690        HES4
      [4]     chr1      959309-1331427      * | 0.0589313       NOC2L
      [5]     chr1      960587-1000388      * | 0.0568306      KLHL17
      ...      ...                 ...    ... .       ...         ...
  [19752]     chrX 154762742-154785456      * | 0.0500833        DKC1
  [19753]     chrX 154763237-155071420      * | 0.0596310       BRCC3
  [19754]     chrX 154776030-154821007      * | 0.0502394        MPP1
  [19755]     chrX 155264589-155334730      * | 0.0517909      RAB39B
  [19756]     chrX 155334657-155334730      * | 0.0609887       CLIC2
                            peak 

In [17]:
df.output = as.data.frame(obj.seurat@assays$ATAC@links)
colnames(df.output) = c('chr',
                        'start',
                        'end',
                        'width',
                        'strand',
                        'score',
                        'TargetGene',
                        'PeakName',
                        'zscore',
                        'pvalue')
df.output[,"CellType"] = "K562"
df.output[,"PairName"] = paste(df.output[,"PeakName"],
                               df.output[,"TargetGene"],
                               sep = "_")
data.table::fwrite(df.output,
                   file = paste(dir.output,"pairs.E2G.res.tsv.gz",sep = "/"),
                   row.names = F,
                   quote = F,
                   sep = "\t")
df.output

chr,start,end,width,strand,score,TargetGene,PeakName,zscore,pvalue,CellType,PairName
<fct>,<int>,<int>,<int>,<fct>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>
chr1,820756,1020123,199368,*,0.05255269,AGRN,chr1-819819-821692,2.741494,3.058026e-03,K562,chr1-819819-821692_AGRN
chr1,827285,1116361,289077,*,0.05328596,C1orf159,chr1-826642-827928,1.674492,4.701694e-02,K562,chr1-826642-827928_C1orf159
chr1,940540,1000172,59633,*,0.05136896,HES4,chr1-939995-941085,1.745786,4.042406e-02,K562,chr1-939995-941085_HES4
chr1,959309,1331427,372119,*,0.05893129,NOC2L,chr1-1330543-1332311,4.254734,1.046487e-05,K562,chr1-1330543-1332311_NOC2L
chr1,960587,1000388,39802,*,0.05683056,KLHL17,chr1-999557-1001218,2.182127,1.455010e-02,K562,chr1-999557-1001218_KLHL17
chr1,960587,1308322,347736,*,0.05114217,KLHL17,chr1-1307532-1309112,2.062394,1.958512e-02,K562,chr1-1307532-1309112_KLHL17
chr1,960857,1000172,39316,*,0.05233106,HES4,chr1-960264-961450,2.140895,1.614124e-02,K562,chr1-960264-961450_HES4
chr1,966713,1000172,33460,*,0.05664950,HES4,chr1-965905-967520,2.003081,2.258431e-02,K562,chr1-965905-967520_HES4
chr1,966713,1399328,432616,*,0.07269854,CCNL2,chr1-965905-967520,2.622200,4.368208e-03,K562,chr1-965905-967520_CCNL2


In [18]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux 8.9 (Ootpa)

Matrix products: default
BLAS/LAPACK: /maps/projects/ralab/people/lpm537/software/anaconda3/envs/Notebook_E2G_240505/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Copenhagen
tzcode source: system (glibc)

attached base packages:
 [1] parallel  stats4    grid      stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] foreach_1.5.2                     Matrix_1.6-5                     
 [3] EnsDb.Hsapiens.v86_2.99.0         ensembldb_2